# Model Definition and Evaluation
## Table of Contents
1. [Model Selection](#model-selection)
2. [Feature Engineering](#feature-engineering)
3. [Hyperparameter Tuning](#hyperparameter-tuning)
4. [Implementation](#implementation)
5. [Evaluation Metrics](#evaluation-metrics)
6. [Comparative Analysis](#comparative-analysis)


In [2]:
#reset all variables when running again to avoid any mistakes

%reset -f

In [3]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import compute_class_weight

import gc
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image

import tensorflow as tf

from keras.src.legacy.preprocessing.image import ImageDataGenerator

from matplotlib.ticker import StrMethodFormatter


## Model Selection

We are considering and evaluating self-defined CNN and pretrained CNNs. Convolutional Neural Networks are proven to work well for the task of image classification. Most likely transformer models could potentially have an even better performance but it is highly likely that the added complexity will not justify the potential increase in performance.



## Hyperparameters


In [4]:
# choose main hyperparameters here

#data / feature selections
balanced_flag = False

#Traing data splits :
test_split = 0.2
val_split = 0.20 # remember - this is fractional  after the test data has been split from the initial balanced sub dataset

# image parameters
target_size = (299,299) #pixel size to load img

#select data augmentation
aug_flag = True
#augmentation params
horizontal_flip=False
vertical_flip=False
rotation_range=15
shear_range= 1
zoom_range = 0.07

#training
max_epochs = 100
loss_stop_patience = 7
learningRate = 0.001

loss_stop_patience_multi = [5,6,12] #patience in each step

#class weights
Use_class_weights = True

#early stopping
StoppingSelector = 'val_loss' #valid values: 'val_loss','val_f1'

#flags for models to include
model1_flag = False
model2_flag = False
model3_flag = False
feature_extract_flag = False
fine_tune_flag = False
full_fine_tune_flag = False
multi_phase_fine_tune_flag = False

reducedClassNumber_flag = False

EfficientNet_Flag = True

#set batch size according to balanc selection
if balanced_flag:
    batch_size = 8 #later player around with batch size to see how it affects performance
else:
    batch_size = 32 #hopefully speeds up training

In [5]:
# specify the model loss function

#options: 'normal' :   tf.keras.losses.CategoricalCrossentropy()    , 'focal' : tf.keras.losses.CategoricalCrossentropy()

lossSelect = 'normal'


In [6]:
#select optimizer

optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate)

In [7]:
#make customizable string to add to dir for testing

custom_save_str = '_focalLoss'

## Feature Engineering

[Describe any additional feature engineering you've performed beyond what was done for the baseline model.]

We test data augmentation (to varying degrees), dataset balancing and class weights.


In [8]:
# Path Definitions to relevant data + data loading

base_file_path = 'C:/Users/nikoLocal/Documents/Opencampus/Machine_Vision_challenge_data/'
image_path = base_file_path + '/input_train/input_train'

label_csv_name = 'Y_train_eVW9jym.csv'

#Loading .csv data to dataframes
train_df = pd.read_csv(os.path.join(base_file_path, label_csv_name))


In [9]:
#DataFrame Preprocessing


#add another column to the dataframe according to dictionaries to map Labels correctly to numbers
dict_numbers = {'GOOD': 0,'Boucle plate':1,'Lift-off blanc':2,'Lift-off noir':3,'Missing':4,'Short circuit MOS':5}
dict_strings = {'GOOD': '0_GOOD','Boucle plate':'1_Flat loop','Lift-off blanc':'2_White lift-off','Lift-off noir':'3_Black lift-off','Missing':'4_Missing','Short circuit MOS':'5_Short circuit MOS'}
# for Test Data ("random submission" dataframe)
dict_strings_sub = {0: '0_GOOD',1:'1_Flat loop',2:'2_White lift-off',3:'3_Black lift-off',4:'4_Missing',5:'5_Short circuit MOS',6:'6_Drift'}

#list of all labels in the data
label_list = ['0_GOOD','1_Flat loop','2_White lift-off','3_Black lift-off','4_Missing','5_Short circuit MOS']

#create new columns in DFs via .map() method
train_df['LabelNum'] = train_df['Label'].map(dict_numbers)
train_df['LabelStr'] = train_df['Label'].map(dict_strings)

#number of classes
num_classes = len(label_list)

# get counts of label with the least entries
countList = train_df['LabelStr'].value_counts()
minCounts = countList.min()

BalancedDF = pd.DataFrame()
#concat sampled dataframes for each included label
for i in range(num_classes):
    BalancedDF = pd.concat([BalancedDF,train_df[train_df['LabelStr'] == label_list[i]].sample(n=minCounts)],axis=0)

#split dataframe according to fractional test size
train_df_balanced, test_df_balanced = train_test_split(BalancedDF, test_size=test_split, random_state=42) #keep random state constant to ensure

train_df_train, train_df_test = train_test_split(train_df, test_size=test_split, random_state=42) #keep random state constant to ensure

In [10]:
#test if worked as intended
print('Balanced DF Label Counts:')
print(BalancedDF['LabelStr'].value_counts())

Balanced DF Label Counts:
LabelStr
0_GOOD                 71
1_Flat loop            71
2_White lift-off       71
3_Black lift-off       71
4_Missing              71
5_Short circuit MOS    71
Name: count, dtype: int64


In [11]:
#test if worked as intended
print('DF Label Counts:')
print(train_df['LabelStr'].value_counts())

DF Label Counts:
LabelStr
4_Missing              6472
0_GOOD                 1235
2_White lift-off        270
5_Short circuit MOS     126
3_Black lift-off        104
1_Flat loop              71
Name: count, dtype: int64


In [12]:
#compute class weights for use of unbalanced datasets

class_numbers = np.unique(train_df_train['LabelNum'])

class_weights_unb = compute_class_weight(class_weight='balanced' ,classes = class_numbers,y=train_df_train['LabelNum'])
class_weights_b = compute_class_weight(class_weight='balanced' ,classes = class_numbers,y=train_df_balanced['LabelNum'])
equal_weights = np.ones(num_classes)

#make dicts that can be used by keras
class_w_unb_dict = dict(zip(class_numbers, class_weights_unb))
class_w_b_dict = dict(zip(class_numbers, class_weights_b))
class_w_equal_dict = dict(zip(class_numbers, equal_weights))

In [13]:
# make a dataset with only labels: 0_Good, 1_Flat loop and 2_Defective
# in order to test whether flat loop can be recognized with high precision by a NN

#make a deepcopy of existing df
df_reduced_labels = train_df.copy(deep = True)

#dict_strings = {'GOOD': '0_GOOD','Boucle plate':'1_Flat loop','Lift-off blanc':'2_White lift-off','Lift-off noir':'3_Black lift-off','Missing':'4_Missing','Short circuit MOS':'5_Short circuit MOS'}

df_reduced_labels.loc[df_reduced_labels['LabelStr'] == '2_White lift-off', 'LabelStr'] = '2_Defective'
df_reduced_labels.loc[df_reduced_labels['LabelStr'] == '3_Black lift-off', 'LabelStr'] = '2_Defective'
df_reduced_labels.loc[df_reduced_labels['LabelStr'] == '4_Missing', 'LabelStr'] = '2_Defective'
df_reduced_labels.loc[df_reduced_labels['LabelStr'] == '5_Short circuit MOS', 'LabelStr'] = '2_Defective'

df_reduced_labels.loc[df_reduced_labels['LabelNum'] == 2, 'LabelNum'] = 2
df_reduced_labels.loc[df_reduced_labels['LabelNum'] == 3, 'LabelNum'] = 2
df_reduced_labels.loc[df_reduced_labels['LabelNum'] == 4, 'LabelNum'] = 2
df_reduced_labels.loc[df_reduced_labels['LabelNum'] == 5, 'LabelNum'] = 2

# split into training and test data
df_reduced_labels_train, df_reduced_labels_test = train_test_split(df_reduced_labels, test_size=test_split, random_state=42) #keep random state constant to ensure

#class weights for this dataset
class_numbers_redLabel = np.unique(df_reduced_labels_train['LabelNum'])

class_weights_redLabel = compute_class_weight(class_weight='balanced' ,classes = class_numbers_redLabel,y=df_reduced_labels_train['LabelNum'])

#make dicts that can be used by keras
class_weights_redLabel_dict = dict(zip(class_numbers_redLabel, class_weights_redLabel))

num_classes_redLabel = 3

label_list_reduced = ['0_Good','1_Flat Loop','2_Defective']

In [14]:
df_reduced_labels['LabelStr'].value_counts()

LabelStr
2_Defective    6972
0_GOOD         1235
1_Flat loop      71
Name: count, dtype: int64

## Hyperparameter Tuning

[Discuss any hyperparameter tuning methods you've applied, such as Grid Search or Random Search, and the rationale behind them.]
So far we have done hyperparameters variation "by hand" only. Some parameters, such as the image size and data augmentation have been systemarically varied and the effects on model performance noted.

We plan to do more hyperparamter tuning in the future.

In [15]:
# make a unique string (name) to save model and evaluation to file
# incorporate most important hyperparameters
# make a subfolder for one set of hyperparameters for more tidy folder and file structure

if aug_flag:
    augmentation_str = 'Aug'
else:
    augmentation_str = 'NoAug'

if balanced_flag:
    balance_str = 'balanced'
else:
    balance_str = 'unbalanced'

#class weights
if Use_class_weights:
    Cweights_str = '_Cweights'
else:
    Cweights_str = ''

hyperparam_name = 'ImgSz_{}_{}_{}{}{}'.format(target_size[0],augmentation_str,balance_str,Cweights_str,custom_save_str)
hyperparam_dir = os.path.join(base_file_path,'model_evaluation')
hyperparam_dir = os.path.join(hyperparam_dir,hyperparam_name)
#check if folder exists - if not create it
if not os.path.isdir(hyperparam_dir):
    os.makedirs(hyperparam_dir)

In [16]:
# initialize ImageDataGenerators
# use ImageDataGen because it has method flow_from_dataframe() that works really well together with pandas dataframes
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
# although deprecated the functionality can be used as discussed in feedback session

# HYPERPARAMTERS ########


class_mode = 'categorical' # how to store labels - either categorical (one-hot encoding) or as numbers
#class_mode = 'input'
labelCol = 'LabelStr'
#########################

#normalize pixel intensities
rescale = 1.0/255.0

datagen = ImageDataGenerator(
    horizontal_flip=False,
    vertical_flip=False,
    rotation_range=0.0,
    shear_range=0.0,
    rescale=rescale,
    validation_split=val_split)

datagen_augmentation = ImageDataGenerator(
    horizontal_flip=horizontal_flip,
    vertical_flip=vertical_flip,
    rotation_range=rotation_range,
    shear_range= shear_range,
    zoom_range = zoom_range,
    rescale=rescale,
    validation_split=val_split)

datagen_test = ImageDataGenerator(
    horizontal_flip=False,
    vertical_flip=False,
    rotation_range=0.0,
    shear_range=0.0,
    rescale=rescale,
    validation_split=0.0)


##########################################################

#unbalanced datasets

train_generator_unbalanced = datagen.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_unbalanced_val = datagen.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='validation')

train_generator_unbalanced_aug = datagen_augmentation.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_unbalanced_val_aug = datagen_augmentation.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='validation')

test_generator_unbalanced = datagen_test.flow_from_dataframe(
    train_df_test,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

test_generator_unbalanced_metrics = datagen_test.flow_from_dataframe(
    train_df_test,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=1,
    color_mode="grayscale",
    shuffle=False,
    seed=42,
    subset='training')

train_generator = datagen.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_val = datagen.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='validation')

train_generator_aug = datagen_augmentation.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_aug_val = datagen_augmentation.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='validation')

test_generator = datagen_test.flow_from_dataframe(
    test_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="grayscale",
    shuffle=True,
    seed=42,
    subset='training')

test_generator_metrics = datagen_test.flow_from_dataframe(
    test_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=1,
    color_mode="grayscale",
    shuffle=False,
    seed=42,
    subset='training')

# generators for transfer learning - color mode is color here. Pretrained models expect color input

train_generator_unbalanced_color = datagen.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_unbalanced_val_color = datagen.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='validation')

test_generator_unbalanced_color = datagen_test.flow_from_dataframe(
    train_df_test,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_unbalanced_aug_color = datagen_augmentation.flow_from_dataframe(
    train_df_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

test_generator_unbalanced_metrics_color = datagen_test.flow_from_dataframe(
    train_df_test,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb",
    shuffle=False,
    seed=42,
    subset='training')

train_generator_color = datagen.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_val_color = datagen.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='validation')

train_generator_aug_color = datagen_augmentation.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_aug_val_color = datagen_augmentation.flow_from_dataframe(
    train_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='validation')

test_generator_color = datagen_test.flow_from_dataframe(
    test_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

test_generator_metrics_color = datagen_test.flow_from_dataframe(
    test_df_balanced,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb",
    shuffle=False,
    seed=42,
    subset='training')


Found 5298 validated image filenames belonging to 6 classes.
Found 1324 validated image filenames belonging to 6 classes.
Found 5298 validated image filenames belonging to 6 classes.
Found 1324 validated image filenames belonging to 6 classes.
Found 1656 validated image filenames belonging to 6 classes.
Found 1656 validated image filenames belonging to 6 classes.
Found 272 validated image filenames belonging to 6 classes.
Found 68 validated image filenames belonging to 6 classes.
Found 272 validated image filenames belonging to 6 classes.
Found 68 validated image filenames belonging to 6 classes.
Found 86 validated image filenames belonging to 6 classes.
Found 86 validated image filenames belonging to 6 classes.
Found 5298 validated image filenames belonging to 6 classes.
Found 1324 validated image filenames belonging to 6 classes.
Found 1656 validated image filenames belonging to 6 classes.
Found 5298 validated image filenames belonging to 6 classes.
Found 1656 validated image filenam

In [ ]:
#dataset for reduced class number

train_generator_redLabel = datagen.flow_from_dataframe(
    df_reduced_labels_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_redLabel_val = datagen.flow_from_dataframe(
    df_reduced_labels_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='validation')

test_generator_redLabel = datagen_test.flow_from_dataframe(
    df_reduced_labels_test,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

train_generator_redLabel_aug = datagen_augmentation.flow_from_dataframe(
    df_reduced_labels_train,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    color_mode="rgb",
    shuffle=True,
    seed=42,
    subset='training')

test_generator_redLabel_metrics = datagen_test.flow_from_dataframe(
    df_reduced_labels_test,
    image_path,
    x_col='filename',
    y_col=labelCol,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=1,
    color_mode="rgb",
    shuffle=False,
    seed=42,
    subset='training')

Found 5298 validated image filenames belonging to 3 classes.
Found 1324 validated image filenames belonging to 3 classes.


## Implementation

[Implement the final model(s) you've selected based on the above steps.]


In [484]:
# build a model to be used as baseline model
# use "simplest" CNN as baseline

model_1_CNN = tf.keras.Sequential([
    tf.keras.layers.Input((target_size[0], target_size[1], 1)),  #image are greyscale - so in total dim (width,height,1)
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(num_classes, activation="softmax"),
])

model_2_CNN = tf.keras.Sequential([
    tf.keras.layers.Input((target_size[0], target_size[1], 1)),  #image are greyscale - so in total dim (width,height,1)
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(num_classes, activation="softmax"),
])

model_3_CNN = tf.keras.Sequential([
    tf.keras.layers.Input((target_size[0], target_size[1], 1)),  #image are greyscale - so in total dim (width,height,1)
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPool2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(num_classes, activation="softmax"),
])

model_1_CNN.summary()
model_2_CNN.summary()
model_3_CNN.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 297, 297, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 148, 148, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1401856)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    89,718,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,719,878 (342.25 MB)

 Trainable params: 89,719,878 (342.25 MB)

 Non-trainable params: 0 (0.00 B)

...

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 297, 297, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 148, 148, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 146, 146, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 73, 73, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 71, 71, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 35, 35, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 156800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    20,070,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,171,846 (76.95 MB)

 Trainable params: 20,171,846 (76.95 MB)

 Non-trainable params: 0 (0.00 B)

In [485]:
#loss

if balanced_flag:
    focal_alpha = class_weights_b
else:
    focal_alpha = class_weights_unb

# callback that monitors validation accuracy / loss
# https://keras.io/api/callbacks/early_stopping/


match StoppingSelector:
    case 'val_loss':
        StopCallback = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            min_delta=0.01,
            patience= loss_stop_patience,
            restore_best_weights=True,
            verbose = 2,
            start_from_epoch = 1
        )
    case 'val_f1':
        StopCallback = tf.keras.callbacks.EarlyStopping(
            monitor='val_f1_score',
            min_delta=0.005,
            patience= loss_stop_patience,
            restore_best_weights=True,
            verbose = 2,
            mode='max'
        )

class ResetValLossOnTrainBegin(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        #set val loss to a high value in case there is a history left
        logs["val_loss"] =  1e3


In [486]:
# load weights gained by training from medical MRI data into inception V3

# Load pre-trained InceptionV3 with correct input size

#path to weights
weights_subpath = 'pre_trained_models/radiology_net/InceptionV3.pth'
medical_weights_path = os.path.join(base_file_path,weights_subpath)

#convert weights to keras format

#base_transfer_model_medical = tf.keras.applications.InceptionV3(
#    weights=medical_weights_path,
#    include_top=False,
#    input_shape=(target_size[0], target_size[0], 3)
#)


In [487]:
#use the custom library from the weights provider
#https://github.com/AIlab-RITEH/RadiologyNET-TL-models

#import models
#import torch

#model_med_torch = models.InceptionV3(pretrained=False, number_of_classes=6)
#models.transfer_weights_to_model(path=medical_weights_path, target_model=model_med_torch, device='cpu')

#test
#onnx_program = torch.onnx.export(model_med_torch, dynamo=True)

In [488]:
#Transfer learning model - feature extraction

# Load pre-trained InceptionV3 with correct input size
base_transfer_model = tf.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(target_size[0], target_size[0], 3)
)

# Freeze all layers for feature extraction
base_transfer_model.trainable = False

# Simple classification head
# - GlobalAveragePooling2D reduces spatial dimensions
# - Final Dense layer maps to class probabilities
inception_feature_extraction_model = tf.keras.Sequential([
    base_transfer_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

#select based on Str
if lossSelect == 'normal':
    loss_fun_feat = tf.keras.losses.CategoricalCrossentropy()
else:
    loss_fun_feat = tf.keras.losses.CategoricalFocalCrossentropy(alpha = focal_alpha,gamma = 2)

inception_feature_extraction_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learningRate),
    loss=loss_fun_feat,
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    #metrics=["accuracy",'precision',]
    weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
)

inception_feature_extraction_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 8, 8, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 6)              │        12,294 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,815,078 (83.22 MB)

 Trainable params: 12,294 (48.02 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [489]:
# inception model with last 20 layers unfrozen

# Load pre-trained InceptionV3 with correct input size
base_transfer_model_2 = tf.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(target_size[0], target_size[0], 3)
)

# Freeze all layers except last few blocks
base_transfer_model_2.trainable = True
for layer in base_transfer_model_2.layers[:-20]:
    layer.trainable = False

# Simple classification head
# - GlobalAveragePooling2D reduces spatial dimensions
# - Final Dense layer maps to class probabilities
inception_fine_tune_model = tf.keras.Sequential([
    base_transfer_model_2,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

#select based on Str
if lossSelect == 'normal':
    loss_fun_fine = tf.keras.losses.CategoricalCrossentropy()
else:
    loss_fun_fine = tf.keras.losses.CategoricalFocalCrossentropy(alpha = focal_alpha,gamma = 2)

inception_fine_tune_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss_fun_fine,
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    #metrics=["accuracy",'precision',]
    weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
)

inception_fine_tune_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 8, 8, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 6)              │        12,294 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,815,078 (83.22 MB)

 Trainable params: 1,947,654 (7.43 MB)

 Non-trainable params: 19,867,424 (75.79 MB)

In [490]:
# inception model with all layers unfrozen

# Load pre-trained InceptionV3 with correct input size
base_transfer_model_3 = tf.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(target_size[0], target_size[0], 3)
)

# Freeze all layers except last few blocks
base_transfer_model_3.trainable = True
#for layer in base_transfer_model_3.layers[:-20]:
#    layer.trainable = False

# Simple classification head
# - GlobalAveragePooling2D reduces spatial dimensions
# - Final Dense layer maps to class probabilities
inception_full_fine_tune_model = tf.keras.Sequential([
    base_transfer_model_3,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2), #do I need to keep this ?
    tf.keras.layers.Dense(num_classes, activation = 'softmax')
])

#select based on Str
if lossSelect == 'normal':
    loss_fun_full_fine = tf.keras.losses.CategoricalCrossentropy()
else:
    loss_fun_full_fine = tf.keras.losses.CategoricalFocalCrossentropy(alpha = focal_alpha,gamma = 2)

inception_full_fine_tune_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss_fun_full_fine,
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    #metrics=["accuracy",'precision',]
    weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
)

inception_full_fine_tune_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 8, 8, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 6)              │        12,294 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,815,078 (83.22 MB)

 Trainable params: 21,780,646 (83.09 MB)

 Non-trainable params: 34,432 (134.50 KB)

In [491]:
# inception model where layers are sucessively unfrozen during training

# Load pre-trained InceptionV3 with correct input size
base_transfer_model_4 = tf.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(target_size[0], target_size[0], 3)
)

# Freeze all layers except last few blocks
base_transfer_model_4.trainable = False
#for layer in base_transfer_model_3.layers[:-20]:
#    layer.trainable = False

# Simple classification head
# - GlobalAveragePooling2D reduces spatial dimensions
# - Final Dense layer maps to class probabilities
inception_multiPhase_fine_tune_model = tf.keras.Sequential([
    base_transfer_model_4,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2), #do I need to keep this ?
    tf.keras.layers.Dense(num_classes, activation = 'softmax')
])

#select based on Str
if lossSelect == 'normal':
    loss_fun_multi_fine = tf.keras.losses.CategoricalCrossentropy()
else:
    loss_fun_multi_fine = tf.keras.losses.CategoricalFocalCrossentropy(alpha = focal_alpha,gamma = 2)

inception_multiPhase_fine_tune_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), #start with "normal" learning rate
    loss=loss_fun_multi_fine,
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    #metrics=["accuracy",'precision',]
    weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
)

inception_multiPhase_fine_tune_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 8, 8, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 6)              │        12,294 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,815,078 (83.22 MB)

 Trainable params: 12,294 (48.02 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [587]:
# inception model where layers are sucessively unfrozen during training - for reduced class Number

# Load pre-trained InceptionV3 with correct input size
base_transfer_model_5 = tf.keras.applications.InceptionV3(
    weights='imagenet',
    include_top=False,
    input_shape=(target_size[0], target_size[0], 3)
)

# Freeze all layers except last few blocks
base_transfer_model_5.trainable = False
#for layer in base_transfer_model_3.layers[:-20]:
#    layer.trainable = False

# Simple classification head
# - GlobalAveragePooling2D reduces spatial dimensions
# - Final Dense layer maps to class probabilities
inception_multiPhase_redLabel = tf.keras.Sequential([
    base_transfer_model_5,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2), #do I need to keep this ?
    tf.keras.layers.Dense(num_classes_redLabel, activation = 'softmax')
])

#select based on Str
if lossSelect == 'normal':
    loss_fun_multi_redLabel = tf.keras.losses.CategoricalCrossentropy()
else:
    loss_fun_multi_redLabel = tf.keras.losses.CategoricalFocalCrossentropy(alpha = focal_alpha,gamma = 2)

inception_multiPhase_redLabel.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), #start with "normal" learning rate
    loss=loss_fun_multi_redLabel,
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    #metrics=["accuracy",'precision',]
    weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
)

inception_multiPhase_redLabel.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 8, 8, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         6,147 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,808,931 (83.19 MB)

 Trainable params: 6,147 (24.01 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [1]:
# EfficientNet model where layers are sucessively unfrozen during training

# Load pre-trained InceptionV3 with correct input size
base_model_efficientNet = tf.keras.applications.EfficientNetV2S(
    weights='imagenet',
    include_top=False,
    include_preprocessing=True,
    input_shape=(target_size[0], target_size[0], 3)
)

# Freeze all layers except last few blocks
base_model_efficientNet.trainable = False
#for layer in base_transfer_model_3.layers[:-20]:
#    layer.trainable = False

# Simple classification head
# - GlobalAveragePooling2D reduces spatial dimensions
# - Final Dense layer maps to class probabilities
EfficientNet_multiPhase_model = tf.keras.Sequential([
    base_model_efficientNet,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2), #do I need to keep this ?
    tf.keras.layers.Dense(num_classes, activation = 'softmax')
])

#select based on Str
if lossSelect == 'normal':
    loss_fun_EfficientNet = tf.keras.losses.CategoricalCrossentropy()
else:
    loss_fun_EfficientNet = tf.keras.losses.CategoricalFocalCrossentropy(alpha = focal_alpha,gamma = 2)

inception_multiPhase_fine_tune_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), #start with "normal" learning rate
    loss=loss_fun_EfficientNet,
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    #metrics=["accuracy",'precision',]
    weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
)

EfficientNet_multiPhase_model.summary()

NameError: name 'tf' is not defined

In [492]:
# compile Models

#select based on Str
if lossSelect == 'normal':
    loss_fun_1 = tf.keras.losses.CategoricalCrossentropy()
else:
    loss_fun_1 = tf.keras.losses.CategoricalFocalCrossentropy(alpha = focal_alpha,gamma = 2)

model_1_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learningRate),
    loss=loss_fun_1,
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    #metrics=["accuracy",'precision',]
    weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
)

#select based on Str
if lossSelect == 'normal':
    loss_fun_2 = tf.keras.losses.CategoricalCrossentropy()
else:
    loss_fun_2 = tf.keras.losses.CategoricalFocalCrossentropy(alpha = focal_alpha,gamma = 2)

model_2_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learningRate),
    loss=loss_fun_2,
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    #metrics=["accuracy",'precision',]
    weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
)

#select based on Str
if lossSelect == 'normal':
    loss_fun_3 = tf.keras.losses.CategoricalCrossentropy()
else:
    loss_fun_3 = tf.keras.losses.CategoricalFocalCrossentropy(alpha = focal_alpha,gamma = 2)

model_3_CNN.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learningRate),
    loss=loss_fun_3,
    #loss=tf.keras.losses.SparseCategoricalCrossentropy,
    #metrics=["accuracy",'precision',]
    #metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
    weighted_metrics =["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
)

#F1 average parameter needs to be anything other than None if using linewise output when fiting the model...



In [493]:
#select model datasets based on flags

#for model1 - 3
if balanced_flag:
    CNN_model_val_gen = train_generator_val
    if aug_flag:
        CNN_model_gen = train_generator_aug
    else:
        CNN_model_gen = train_generator
    #class weights
    class_weights_training = class_weights_b
else:
    CNN_model_val_gen = train_generator_unbalanced_val
    if aug_flag:
        CNN_model_gen = train_generator_unbalanced_aug
    else:
        CNN_model_gen = train_generator_unbalanced

#for transfer learning
if balanced_flag:
    transfer_model_val_gen = train_generator_val_color
    if aug_flag:
        transfer_model_gen = train_generator_aug_color
    else:
        transfer_model_gen = train_generator_color
else:
    transfer_model_val_gen = train_generator_unbalanced_val_color
    if aug_flag:
        transfer_model_gen = train_generator_unbalanced_aug_color
    else:
        transfer_model_gen = train_generator_unbalanced_color


if Use_class_weights:
    if balanced_flag:
        class_weights_training = class_w_b_dict
    else:
        class_weights_training = class_w_unb_dict
else:
    class_weights_training = class_w_equal_dict


In [494]:

if model1_flag:

    history_1 = model_1_CNN.fit(
    CNN_model_gen,
    validation_data = CNN_model_val_gen,
    epochs=max_epochs,
    class_weight = class_weights_training,
    callbacks=[StopCallback,ResetValLossOnTrainBegin()],
    verbose = 2 #2 is one line per epoch -
    )

In [495]:
#Model 2

if model2_flag:
    history_2 = model_2_CNN.fit(
        CNN_model_gen,
        validation_data = CNN_model_val_gen,
        epochs=max_epochs,
        class_weight = class_weights_training,
        callbacks=[StopCallback,ResetValLossOnTrainBegin()],
        verbose = 2 #2 is one line per epoch -
    )

In [496]:
#Model 3

if model3_flag:
    # model_3_CNN.fit(
    #     CNN_model_gen,
    #     validation_data = CNN_model_val_gen,
    #     epochs=3,
    #     class_weight = class_weights_training,
    #     callbacks=[StopCallback,ResetValLossOnTrainBegin()],
    #     verbose = 2 #2 is one line per epoch -
    # )

    history_3 = model_3_CNN.fit(
        CNN_model_gen,
        validation_data = CNN_model_val_gen,
        epochs=max_epochs,
        class_weight = class_weights_training,
        callbacks=[StopCallback,ResetValLossOnTrainBegin()],
        verbose = 2 #2 is one line per epoch -
    )

In [497]:
#transfer learning. Feature extraction

if feature_extract_flag:
    history_feat_extract = inception_feature_extraction_model.fit(
        transfer_model_gen,
        validation_data = transfer_model_val_gen,
        epochs=max_epochs,
        class_weight = class_weights_training,
        callbacks=[StopCallback,ResetValLossOnTrainBegin()],
        verbose = 2 #2 is one line per epoch -
    )

In [498]:
#transfer learning. fine tuning

if fine_tune_flag:
    # inception_fine_tune_model.fit(
    #     transfer_model_gen,
    #     validation_data = transfer_model_val_gen,
    #     epochs=3,
    #     class_weight = class_weights_training,
    #     callbacks=[StopCallback,ResetValLossOnTrainBegin()],
    #     verbose = 2 #2 is one line per epoch -
    # )

    history_fine_tune = inception_fine_tune_model.fit(
        transfer_model_gen,
        validation_data = transfer_model_val_gen,
        epochs=max_epochs,
        class_weight = class_weights_training,
        callbacks=[StopCallback,ResetValLossOnTrainBegin()],
        verbose = 2 #2 is one line per epoch -
    )

In [499]:
# transfer learning. fine tuning of full model
# propably make a scheduler for the learning rate.
# Also - train model sucessively

if full_fine_tune_flag:
    # inception_fine_tune_model.fit(
    #     transfer_model_gen,
    #     validation_data = transfer_model_val_gen,
    #     epochs=3,
    #     class_weight = class_weights_training,
    #     callbacks=[StopCallback,ResetValLossOnTrainBegin()],
    #     verbose = 2 #2 is one line per epoch -
    # )

    history_full_fine_tune = inception_full_fine_tune_model.fit(
        transfer_model_gen,
        validation_data = transfer_model_val_gen,
        epochs=max_epochs,
        class_weight = class_weights_training,
        callbacks=[StopCallback,ResetValLossOnTrainBegin()],
        verbose = 2 #2 is one line per epoch -
    )

In [500]:
# transfer learning. fine tuning of full model
# multi phase fine tuning

#delete history if it already exists to unsure expected local stopping behaviour
if 'history_multi_fine_tune' in locals():
    del history_multi_fine_tune

if multi_phase_fine_tune_flag:

    match StoppingSelector:
        case 'val_loss':
            StopCallback_multi = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0.01,
                patience= loss_stop_patience_multi[0],
                restore_best_weights=True,
                verbose = 2,
                start_from_epoch = 1
            )
        case 'val_f1':
            StopCallback_multi = tf.keras.callbacks.EarlyStopping(
                monitor='val_f1_score',
                min_delta=0.005,
                patience= loss_stop_patience_multi[0],
                restore_best_weights=True,
                verbose = 2,
                mode='max'
            )

    # train last layer first
    history_multi_fine_tune = inception_multiPhase_fine_tune_model.fit(
        transfer_model_gen,
        validation_data = transfer_model_val_gen,
        epochs=max_epochs,
        class_weight = class_weights_training,
        callbacks=[StopCallback_multi],
        verbose = 2 #2 is one line per epoch -
    )

Epoch 1/100
166/166 - 285s - 2s/step - accuracy: 0.5636 - f1_score: 0.5589 - loss: 1.1622 - precision: 0.7397 - recall: 0.3864 - val_accuracy: 0.9245 - val_f1_score: 0.9371 - val_loss: 0.2092 - val_precision: 0.9627 - val_recall: 0.8965
Epoch 2/100
166/166 - 245s - 1s/step - accuracy: 0.7637 - f1_score: 0.7631 - loss: 0.6516 - precision: 0.8191 - recall: 0.6805 - val_accuracy: 0.5627 - val_f1_score: 0.6664 - val_loss: 1.2084 - val_precision: 0.5729 - val_recall: 0.4275
Epoch 3/100
166/166 - 244s - 1s/step - accuracy: 0.8291 - f1_score: 0.8285 - loss: 0.5151 - precision: 0.8657 - recall: 0.7511 - val_accuracy: 0.9698 - val_f1_score: 0.9713 - val_loss: 0.1221 - val_precision: 0.9815 - val_recall: 0.9607
Epoch 4/100
166/166 - 244s - 1s/step - accuracy: 0.8357 - f1_score: 0.8356 - loss: 0.4729 - precision: 0.8791 - recall: 0.7994 - val_accuracy: 0.9690 - val_f1_score: 0.9719 - val_loss: 0.0863 - val_precision: 0.9808 - val_recall: 0.9653
Epoch 5/100
166/166 - 245s - 1s/step - accuracy: 0.8

In [501]:
if multi_phase_fine_tune_flag:
    #delete old history to avoid early stopping unexpected behaviour
    del history_multi_fine_tune

    #set last 30 layers to be trainable
    for layer in base_transfer_model_4.layers[-30:]:
        layer.trainable = True

    match StoppingSelector:
        case 'val_loss':
            StopCallback_multi = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0.01,
                patience= loss_stop_patience_multi[1],
                restore_best_weights=True,
                verbose = 2,
                start_from_epoch = 1
            )
        case 'val_f1':
            StopCallback_multi = tf.keras.callbacks.EarlyStopping(
                monitor='val_f1_score',
                min_delta=0.005,
                patience= loss_stop_patience_multi[1],
                restore_best_weights=True,
                verbose = 2,
                mode='max'
            )

    inception_multiPhase_fine_tune_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), #go to lower learning rate
        loss=loss_fun_multi_fine,
        #loss=tf.keras.losses.SparseCategoricalCrossentropy,
        #metrics=["accuracy",'precision',]
        weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
    )

    history_multi_fine_tune = inception_multiPhase_fine_tune_model.fit(
        transfer_model_gen,
        validation_data = transfer_model_val_gen,
        epochs=max_epochs,
        class_weight = class_weights_training,
        callbacks=[StopCallback_multi,ResetValLossOnTrainBegin()],
        verbose = 2 #2 is one line per epoch -
    )

Epoch 1/100
166/166 - 297s - 2s/step - accuracy: 0.7466 - f1_score: 0.7460 - loss: 0.8215 - precision: 0.7688 - recall: 0.7232 - val_accuracy: 0.9766 - val_f1_score: 0.9803 - val_loss: 0.0759 - val_precision: 0.9780 - val_recall: 0.9728
Epoch 2/100
166/166 - 291s - 2s/step - accuracy: 0.8727 - f1_score: 0.8724 - loss: 0.3748 - precision: 0.8911 - recall: 0.8541 - val_accuracy: 0.9834 - val_f1_score: 0.9841 - val_loss: 0.0509 - val_precision: 0.9871 - val_recall: 0.9796
Epoch 3/100
166/166 - 284s - 2s/step - accuracy: 0.8833 - f1_score: 0.8829 - loss: 0.3150 - precision: 0.8889 - recall: 0.8705 - val_accuracy: 0.9902 - val_f1_score: 0.9907 - val_loss: 0.0323 - val_precision: 0.9917 - val_recall: 0.9887
Epoch 4/100
166/166 - 297s - 2s/step - accuracy: 0.9092 - f1_score: 0.9092 - loss: 0.2218 - precision: 0.9170 - recall: 0.9041 - val_accuracy: 0.9879 - val_f1_score: 0.9877 - val_loss: 0.0379 - val_precision: 0.9886 - val_recall: 0.9856
Epoch 5/100
166/166 - 284s - 2s/step - accuracy: 0.9

In [502]:
if multi_phase_fine_tune_flag:

    match StoppingSelector:
        case 'val_loss':
            StopCallback_multi = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0.01,
                patience= loss_stop_patience_multi[2],
                restore_best_weights=True,
                verbose = 2,
                start_from_epoch = 1
            )
        case 'val_f1':
            StopCallback_multi = tf.keras.callbacks.EarlyStopping(
                monitor='val_f1_score',
                min_delta=0.005,
                patience= loss_stop_patience_multi[2],
                restore_best_weights=True,
                verbose = 2,
                mode='max'
            )

    #delete old history to avoid early stopping unexpected behaviour
    del history_multi_fine_tune

    #set last 100 layers to trainable
    for layer in base_transfer_model_4.layers[-100:]:
        layer.trainable = True

    inception_multiPhase_fine_tune_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), #go to lower learning rate
        loss=loss_fun_multi_fine,
        #loss=tf.keras.losses.SparseCategoricalCrossentropy,
        #metrics=["accuracy",'precision',]
        weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
    )

    history_multi_fine_tune = inception_multiPhase_fine_tune_model.fit(
        transfer_model_gen,
        validation_data = transfer_model_val_gen,
        epochs=max_epochs,
        class_weight = class_weights_training,
        callbacks=[StopCallback_multi,ResetValLossOnTrainBegin()],
        verbose = 2 #2 is one line per epoch -
    )

Epoch 1/100
166/166 - 373s - 2s/step - accuracy: 0.8879 - f1_score: 0.8874 - loss: 0.3242 - precision: 0.8966 - recall: 0.8824 - val_accuracy: 0.9864 - val_f1_score: 0.9875 - val_loss: 0.0588 - val_precision: 0.9864 - val_recall: 0.9864
Epoch 2/100
166/166 - 360s - 2s/step - accuracy: 0.9203 - f1_score: 0.9206 - loss: 0.2242 - precision: 0.9226 - recall: 0.9126 - val_accuracy: 0.9902 - val_f1_score: 0.9907 - val_loss: 0.0342 - val_precision: 0.9909 - val_recall: 0.9894
Epoch 3/100
166/166 - 360s - 2s/step - accuracy: 0.9578 - f1_score: 0.9578 - loss: 0.1253 - precision: 0.9602 - recall: 0.9520 - val_accuracy: 0.9864 - val_f1_score: 0.9873 - val_loss: 0.0426 - val_precision: 0.9879 - val_recall: 0.9864
Epoch 4/100
166/166 - 361s - 2s/step - accuracy: 0.9501 - f1_score: 0.9500 - loss: 0.1643 - precision: 0.9508 - recall: 0.9475 - val_accuracy: 0.9751 - val_f1_score: 0.9817 - val_loss: 0.0639 - val_precision: 0.9780 - val_recall: 0.9721
Epoch 5/100
166/166 - 361s - 2s/step - accuracy: 0.9

In [588]:
#train fine tuning model with reduced number of label classes

if 'history_multi_redLabel' in locals():
    del history_multi_redLabel

if reducedClassNumber_flag:

    match StoppingSelector:
        case 'val_loss':
            StopCallback_redLabel = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0.01,
                patience= loss_stop_patience_multi[0],
                restore_best_weights=True,
                verbose = 2,
                start_from_epoch = 1
            )
        case 'val_f1':
            StopCallback_redLabel = tf.keras.callbacks.EarlyStopping(
                monitor='val_f1_score',
                min_delta=0.005,
                patience= loss_stop_patience_multi[0],
                restore_best_weights=True,
                verbose = 2,
                mode='max'
            )

    # train last layer first
    history_multi_redLabel = inception_multiPhase_redLabel.fit(
        train_generator_redLabel_aug,
        validation_data = train_generator_redLabel_val,
        epochs=max_epochs,
        class_weight = class_weights_redLabel_dict,
        callbacks=[StopCallback_redLabel],
        verbose = 2 #2 is one line per epoch -
    )

Epoch 1/100
166/166 - 280s - 2s/step - accuracy: 0.7742 - f1_score: 0.7741 - loss: 0.6075 - precision: 0.8065 - recall: 0.7583 - val_accuracy: 0.9381 - val_f1_score: 0.9424 - val_loss: 0.1852 - val_precision: 0.9407 - val_recall: 0.9343
Epoch 2/100
166/166 - 269s - 2s/step - accuracy: 0.8846 - f1_score: 0.8846 - loss: 0.3349 - precision: 0.8960 - recall: 0.8786 - val_accuracy: 0.8882 - val_f1_score: 0.9180 - val_loss: 0.2656 - val_precision: 0.8946 - val_recall: 0.8784
Epoch 3/100
166/166 - 248s - 1s/step - accuracy: 0.8923 - f1_score: 0.8924 - loss: 0.2852 - precision: 0.9024 - recall: 0.8872 - val_accuracy: 0.9396 - val_f1_score: 0.9466 - val_loss: 0.2014 - val_precision: 0.9408 - val_recall: 0.9366
Epoch 4/100
166/166 - 249s - 2s/step - accuracy: 0.8944 - f1_score: 0.8945 - loss: 0.2664 - precision: 0.8953 - recall: 0.8916 - val_accuracy: 0.9502 - val_f1_score: 0.9563 - val_loss: 0.1386 - val_precision: 0.9523 - val_recall: 0.9494
Epoch 5/100
166/166 - 249s - 1s/step - accuracy: 0.9

In [589]:
if reducedClassNumber_flag:

    del history_multi_redLabel

    match StoppingSelector:
        case 'val_loss':
            StopCallback_redLabel = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0.01,
                patience= loss_stop_patience_multi[1],
                restore_best_weights=True,
                verbose = 2,
                start_from_epoch = 1
            )
        case 'val_f1':
            StopCallback_redLabel = tf.keras.callbacks.EarlyStopping(
                monitor='val_f1_score',
                min_delta=0.005,
                patience= loss_stop_patience_multi[1],
                restore_best_weights=True,
                verbose = 2,
                mode='max'
            )

    #set last 30 layers to be trainable
    for layer in base_transfer_model_5.layers[-30:]:
        layer.trainable = True

    inception_multiPhase_redLabel.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), #go to lower learning rate
        loss=loss_fun_multi_redLabel,
        weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
    )

    history_multi_redLabel = inception_multiPhase_redLabel.fit(
        train_generator_redLabel_aug,
        validation_data = train_generator_redLabel_val,
        epochs=max_epochs,
        class_weight = class_weights_redLabel_dict,
        callbacks=[StopCallback_redLabel],
        verbose = 2 #2 is one line per epoch -
    )

Epoch 1/100
166/166 - 333s - 2s/step - accuracy: 0.8353 - f1_score: 0.8342 - loss: 0.6487 - precision: 0.8424 - recall: 0.8350 - val_accuracy: 0.9653 - val_f1_score: 0.9628 - val_loss: 0.1204 - val_precision: 0.9653 - val_recall: 0.9653
Epoch 2/100
166/166 - 314s - 2s/step - accuracy: 0.9197 - f1_score: 0.9196 - loss: 0.2680 - precision: 0.9210 - recall: 0.9189 - val_accuracy: 0.9622 - val_f1_score: 0.9651 - val_loss: 0.1113 - val_precision: 0.9622 - val_recall: 0.9607
Epoch 3/100
166/166 - 313s - 2s/step - accuracy: 0.9319 - f1_score: 0.9319 - loss: 0.2434 - precision: 0.9332 - recall: 0.9312 - val_accuracy: 0.9690 - val_f1_score: 0.9714 - val_loss: 0.1161 - val_precision: 0.9697 - val_recall: 0.9683
Epoch 4/100


KeyboardInterrupt: 

In [591]:
if reducedClassNumber_flag:

    #del history_multi_redLabel

    match StoppingSelector:
        case 'val_loss':
            StopCallback_redLabel = tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0.01,
                patience= loss_stop_patience_multi[2],
                restore_best_weights=True,
                verbose = 2,
                start_from_epoch = 1
            )
        case 'val_f1':
            StopCallback_redLabel = tf.keras.callbacks.EarlyStopping(
                monitor='val_f1_score',
                min_delta=0.005,
                patience= loss_stop_patience_multi[2],
                restore_best_weights=True,
                verbose = 2,
                mode='max'
            )

    #set last 30 layers to be trainable
    for layer in base_transfer_model_5.layers[-100:]:
        layer.trainable = True

    inception_multiPhase_redLabel.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), #go to lower learning rate
        loss=loss_fun_multi_redLabel,
        weighted_metrics=["accuracy",'precision','recall',tf.keras.metrics.F1Score(average='weighted')]
    )

    history_multi_redLabel = inception_multiPhase_redLabel.fit(
        train_generator_redLabel_aug,
        validation_data = train_generator_redLabel_val,
        epochs=max_epochs,
        class_weight = class_weights_redLabel_dict,
        callbacks=[StopCallback_redLabel],
        verbose = 2 #2 is one line per epoch -
    )

Epoch 1/100
166/166 - 381s - 2s/step - accuracy: 0.9263 - f1_score: 0.9264 - loss: 0.2290 - precision: 0.9279 - recall: 0.9259 - val_accuracy: 0.9834 - val_f1_score: 0.9837 - val_loss: 0.0401 - val_precision: 0.9834 - val_recall: 0.9826
Epoch 2/100
166/166 - 357s - 2s/step - accuracy: 0.9491 - f1_score: 0.9491 - loss: 0.1529 - precision: 0.9509 - recall: 0.9485 - val_accuracy: 0.9539 - val_f1_score: 0.9564 - val_loss: 0.2458 - val_precision: 0.9539 - val_recall: 0.9539
Epoch 3/100
166/166 - 356s - 2s/step - accuracy: 0.9471 - f1_score: 0.9471 - loss: 0.1522 - precision: 0.9553 - recall: 0.9469 - val_accuracy: 0.9705 - val_f1_score: 0.9731 - val_loss: 0.1004 - val_precision: 0.9705 - val_recall: 0.9705
Epoch 4/100
166/166 - 357s - 2s/step - accuracy: 0.9601 - f1_score: 0.9600 - loss: 0.1114 - precision: 0.9611 - recall: 0.9593 - val_accuracy: 0.9086 - val_f1_score: 0.9425 - val_loss: 0.1403 - val_precision: 0.9093 - val_recall: 0.9086
Epoch 5/100
166/166 - 357s - 2s/step - accuracy: 0.9

## Evaluation Metrics

[Clearly specify which metrics you'll use to evaluate the model performance, and why you've chosen these metrics.]


In [503]:
# Model 2

if model2_flag:

    #make one folder for each model to save metrics
    model_2_dir = os.path.join(hyperparam_dir,'model_2')
    if not os.path.isdir(model_2_dir):
        os.makedirs(model_2_dir)

    # test accuracy on test data
    if balanced_flag:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = model_2_CNN.evaluate(test_generator)

        #for classification report
        true_labels = test_generator_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = model_2_CNN.predict(test_generator_metrics)

    else:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = model_2_CNN.evaluate(test_generator_unbalanced)

        true_labels = test_generator_unbalanced_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = model_2_CNN.predict(test_generator_unbalanced_metrics)

    #convert to numerical - np.argmax directly does the job
    predicted_labels = np.argmax(predicted_labels, axis=-1)

    print(f"Model 2. Test Accuracy: {test_accuracy:.3g} | Test Loss: {test_loss:.3g} | Test Precision: {test_precision:.3g} | Test Recall: {test_recall:.3g} | Test F1 Score: {test_f1_score:.3g}:")

    print(classification_report(true_labels, predicted_labels,target_names = label_list))

    #save as dict for future use as well
    report = classification_report(true_labels, predicted_labels,target_names = label_list,output_dict=True)
    #convert to dataframe for easy use and saving to csv
    report_df = pd.DataFrame(report).transpose()

    #save to file
    metrics_baseline_savename = os.path.join(model_2_dir,'classification_report.csv')

    report_df.to_csv(metrics_baseline_savename)

    #save model as well for future use
    #save the model:
    model_2_CNN.save(os.path.join(model_2_dir,'model.keras'))

In [504]:
# Model 3

if model3_flag:

    #make one folder for each model to save metrics
    model_3_dir = os.path.join(hyperparam_dir,'model_3')
    if not os.path.isdir(model_3_dir):
        os.makedirs(model_3_dir)

    # test accuracy on test data
    if balanced_flag:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = model_3_CNN.evaluate(test_generator)

        #for classification report
        true_labels = test_generator_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = model_3_CNN.predict(test_generator_metrics)

    else:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = model_3_CNN.evaluate(test_generator_unbalanced)

        true_labels = test_generator_unbalanced_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = model_3_CNN.predict(test_generator_unbalanced_metrics)

    #convert to numerical - np.argmax directly does the job
    predicted_labels = np.argmax(predicted_labels, axis=-1)

    print(f"Model 3. Test Accuracy: {test_accuracy:.3g} | Test Loss: {test_loss:.3g} | Test Precision: {test_precision:.3g} | Test Recall: {test_recall:.3g} | Test F1 Score: {test_f1_score:.3g}:")

    print(classification_report(true_labels, predicted_labels,target_names = label_list))

    #save as dict for future use as well
    report = classification_report(true_labels, predicted_labels,target_names = label_list,output_dict=True)
    #convert to dataframe for easy use and saving to csv
    report_df = pd.DataFrame(report).transpose()

    #save to file
    metrics_baseline_savename = os.path.join(model_3_dir,'classification_report.csv')

    report_df.to_csv(metrics_baseline_savename)

    #save model as well for future use
    #save the model:
    model_3_CNN.save(os.path.join(model_3_dir,'model.keras'))

In [505]:
# Model 1

if model1_flag:

    #make one folder for each model to save metrics
    model_1_dir = os.path.join(hyperparam_dir,'model_1')
    if not os.path.isdir(model_1_dir):
        os.makedirs(model_1_dir)

    # test accuracy on test data
    if balanced_flag:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = model_1_CNN.evaluate(test_generator)

        #for classification report
        true_labels = test_generator_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = model_1_CNN.predict(test_generator_metrics)

    else:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = model_1_CNN.evaluate(test_generator_unbalanced)

        true_labels = test_generator_unbalanced_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = model_1_CNN.predict(test_generator_unbalanced_metrics)

    #convert to numerical - np.argmax directly does the job
    predicted_labels = np.argmax(predicted_labels, axis=-1)

    print(f"Model 1. Test Accuracy: {test_accuracy:.3g} | Test Loss: {test_loss:.3g} | Test Precision: {test_precision:.3g} | Test Recall: {test_recall:.3g} | Test F1 Score: {test_f1_score:.3g}:")

    print(classification_report(true_labels, predicted_labels,target_names = label_list))

    #save as dict for future use as well
    report = classification_report(true_labels, predicted_labels,target_names = label_list,output_dict=True)
    #convert to dataframe for easy use and saving to csv
    report_df = pd.DataFrame(report).transpose()

    #save to file
    metrics_baseline_savename = os.path.join(model_1_dir,'classification_report.csv')

    report_df.to_csv(metrics_baseline_savename)

    #save model as well for future use
    #save the model:
    model_1_CNN.save(os.path.join(model_1_dir,'model.keras'))

In [506]:
# Model transfer feature extraction

if feature_extract_flag:

    #make one folder for each model to save metrics
    model_feat_extract_dir = os.path.join(hyperparam_dir,'InceptionV3_feat_extract')
    if not os.path.isdir(model_feat_extract_dir):
        os.makedirs(model_feat_extract_dir)

    # test accuracy on test data
    if balanced_flag:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = inception_feature_extraction_model.evaluate(test_generator_color)

        #for classification report
        true_labels = test_generator_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = inception_feature_extraction_model.predict(test_generator_metrics_color)

    else:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = inception_feature_extraction_model.evaluate(test_generator_unbalanced_color)

        true_labels = test_generator_unbalanced_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = inception_feature_extraction_model.predict(test_generator_unbalanced_metrics_color)

    #convert to numerical - np.argmax directly does the job
    predicted_labels = np.argmax(predicted_labels, axis=-1)

    print(f"Feat. Extract. Model:  Test Accuracy: {test_accuracy:.3g} | Test Loss: {test_loss:.3g} | Test Precision: {test_precision:.3g} | Test Recall: {test_recall:.3g} | Test F1 Score: {test_f1_score:.3g}:")

    print(classification_report(true_labels, predicted_labels,target_names = label_list))

    #save as dict for future use as well
    report = classification_report(true_labels, predicted_labels,target_names = label_list,output_dict=True)
    #convert to dataframe for easy use and saving to csv
    report_df = pd.DataFrame(report).transpose()

    #save to file
    metrics_baseline_savename = os.path.join(model_feat_extract_dir,'classification_report.csv')

    report_df.to_csv(metrics_baseline_savename)

    #save model as well for future use
    #save the model:
    inception_feature_extraction_model.save(os.path.join(model_feat_extract_dir,'model.keras'))

In [507]:
#for tine tuning model

if fine_tune_flag:

    #make one folder for each model to save metrics
    model_feat_extract_dir = os.path.join(hyperparam_dir,'InceptionV3_fine_tune')
    if not os.path.isdir(model_feat_extract_dir):
        os.makedirs(model_feat_extract_dir)

    # test accuracy on test data
    if balanced_flag:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = inception_fine_tune_model.evaluate(test_generator_color)

        #for classification report
        true_labels = test_generator_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = inception_fine_tune_model.predict(test_generator_metrics_color)

    else:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = inception_fine_tune_model.evaluate(test_generator_unbalanced_color)

        true_labels = test_generator_unbalanced_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = inception_fine_tune_model.predict(test_generator_unbalanced_metrics_color)

    #convert to numerical - np.argmax directly does the job
    predicted_labels = np.argmax(predicted_labels, axis=-1)

    print(f"Fine tune Model:  Test Accuracy: {test_accuracy:.3g} | Test Loss: {test_loss:.3g} | Test Precision: {test_precision:.3g} | Test Recall: {test_recall:.3g} | Test F1 Score: {test_f1_score:.3g}:")

    print(classification_report(true_labels, predicted_labels,target_names = label_list))

    #save as dict for future use as well
    report = classification_report(true_labels, predicted_labels,target_names = label_list,output_dict=True)
    #convert to dataframe for easy use and saving to csv
    report_df = pd.DataFrame(report).transpose()

    #save to file
    metrics_baseline_savename = os.path.join(model_feat_extract_dir,'classification_report.csv')

    report_df.to_csv(metrics_baseline_savename)

    #save model as well for future use
    #save the model:
    inception_fine_tune_model.save(os.path.join(model_feat_extract_dir,'model.keras'))

In [508]:
#for full tine tuning model

if full_fine_tune_flag:

    #make one folder for each model to save metrics
    model_full_fine_tune_dir = os.path.join(hyperparam_dir,'InceptionV3_full_fine_tune')
    if not os.path.isdir(model_full_fine_tune_dir):
        os.makedirs(model_full_fine_tune_dir)

    # test accuracy on test data
    if balanced_flag:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = inception_full_fine_tune_model.evaluate(test_generator_color)

        #for classification report
        true_labels = test_generator_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = inception_full_fine_tune_model.predict(test_generator_metrics_color)

    else:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = inception_full_fine_tune_model.evaluate(test_generator_unbalanced_color)

        true_labels = test_generator_unbalanced_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = inception_full_fine_tune_model.predict(test_generator_unbalanced_metrics_color)

    #convert to numerical - np.argmax directly does the job
    predicted_labels = np.argmax(predicted_labels, axis=-1)

    print(f"Full fine tune Model:  Test Accuracy: {test_accuracy:.3g} | Test Loss: {test_loss:.3g} | Test Precision: {test_precision:.3g} | Test Recall: {test_recall:.3g} | Test F1 Score: {test_f1_score:.3g}:")

    print(classification_report(true_labels, predicted_labels,target_names = label_list))

    #save as dict for future use as well
    report = classification_report(true_labels, predicted_labels,target_names = label_list,output_dict=True)
    #convert to dataframe for easy use and saving to csv
    report_df = pd.DataFrame(report).transpose()

    #save to file
    metrics_baseline_savename = os.path.join(model_full_fine_tune_dir,'classification_report.csv')

    report_df.to_csv(metrics_baseline_savename)

    #save model as well for future use
    #save the model:
    inception_full_fine_tune_model.save(os.path.join(model_full_fine_tune_dir,'model.keras'))

In [598]:
#for multi phase full tine tuning model - reduced class number
if reducedClassNumber_flag:

    #make one folder for each model to save metrics
    model_fine_tune_redLabel_dir = os.path.join(hyperparam_dir,'InceptionV3_MultiPhase_reducedLabelNum')
    if not os.path.isdir(model_fine_tune_redLabel_dir):
        os.makedirs(model_fine_tune_redLabel_dir)

    # test accuracy on test data - test accuracy ALWAYS on full test set

    test_loss, test_accuracy, test_precision, test_recall,test_f1_score = inception_multiPhase_redLabel.evaluate(test_generator_redLabel)

    #for classification report
    true_labels = test_generator_redLabel_metrics.classes
    # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
    predicted_labels = inception_multiPhase_redLabel.predict(test_generator_redLabel_metrics)

    #convert to numerical - np.argmax directly does the job
    predicted_labels = np.argmax(predicted_labels, axis=-1)

    print(f"Reduced Label Number:  Test Accuracy: {test_accuracy:.3g} | Test Loss: {test_loss:.3g} | Test Precision: {test_precision:.3g} | Test Recall: {test_recall:.3g} | Test F1 Score: {test_f1_score:.3g}:")

    print(classification_report(true_labels, predicted_labels,target_names = label_list_reduced))

    #save as dict for future use as well
    report = classification_report(true_labels, predicted_labels,target_names = label_list_reduced,output_dict=True)
    #convert to dataframe for easy use and saving to csv
    report_df = pd.DataFrame(report).transpose()

    #save to file
    metrics_baseline_savename = os.path.join(model_fine_tune_redLabel_dir,'classification_report.csv')

    report_df.to_csv(metrics_baseline_savename)

    #save model as well for future use
    #save the model:
    inception_multiPhase_redLabel.save(os.path.join(model_fine_tune_redLabel_dir,'model.keras'))

52/52 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.9873 - f1_score: 0.9882 - loss: 0.0536 - precision: 0.9879 - recall: 0.9873
1656/1656 ━━━━━━━━━━━━━━━━━━━━ 117s 70ms/step
Reduced Label Number:  Test Accuracy: 0.987 | Test Loss: 0.0536 | Test Precision: 0.988 | Test Recall: 0.987 | Test F1 Score: 0.988:
              precision    recall  f1-score   support

      0_Good       0.98      0.99      0.98       238
 1_Flat Loop       0.52      0.80      0.63        15
 2_Defective       1.00      0.99      0.99      1403

    accuracy                           0.99      1656
   macro avg       0.83      0.93      0.87      1656
weighted avg       0.99      0.99      0.99      1656



In [592]:
#for multi phase full tine tuning model

if multi_phase_fine_tune_flag:

    #make one folder for each model to save metrics
    model_multi_full_fine_tune_dir = os.path.join(hyperparam_dir,'InceptionV3_MultiPhase_full_fine_tune')
    if not os.path.isdir(model_multi_full_fine_tune_dir):
        os.makedirs(model_multi_full_fine_tune_dir)

    # test accuracy on test data - test accuracy ALWAYS on full test set
    if balanced_flag:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = inception_multiPhase_fine_tune_model.evaluate(test_generator_color)

        #for classification report
        true_labels = test_generator_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = inception_multiPhase_fine_tune_model.predict(test_generator_metrics_color)

    else:
        test_loss, test_accuracy, test_precision, test_recall,test_f1_score = inception_multiPhase_fine_tune_model.evaluate(test_generator_unbalanced_color)

        true_labels = test_generator_unbalanced_metrics.classes
        # model.predict directly gives you the output of the last mode layer. so percentages when using i.e. 'softmax'
        predicted_labels = inception_multiPhase_fine_tune_model.predict(test_generator_unbalanced_metrics_color)

    #convert to numerical - np.argmax directly does the job
    predicted_labels = np.argmax(predicted_labels, axis=-1)

    print(f"Full fine tune Model:  Test Accuracy: {test_accuracy:.3g} | Test Loss: {test_loss:.3g} | Test Precision: {test_precision:.3g} | Test Recall: {test_recall:.3g} | Test F1 Score: {test_f1_score:.3g}:")

    print(classification_report(true_labels, predicted_labels,target_names = label_list))

    #save as dict for future use as well
    report = classification_report(true_labels, predicted_labels,target_names = label_list,output_dict=True)
    #convert to dataframe for easy use and saving to csv
    report_df = pd.DataFrame(report).transpose()

    #save to file
    metrics_baseline_savename = os.path.join(model_multi_full_fine_tune_dir,'classification_report.csv')

    report_df.to_csv(metrics_baseline_savename)

    #save model as well for future use
    #save the model:
    inception_multiPhase_fine_tune_model.save(os.path.join(model_multi_full_fine_tune_dir,'model.keras'))

In [510]:
#clear all models from memory to prevent any bugs and weird behaviour of early stopping

#see: https://stackoverflow.com/questions/58137677/keras-model-training-memory-leak

del model_1_CNN
del model_2_CNN
del model_3_CNN

del inception_feature_extraction_model
del base_transfer_model
del inception_fine_tune_model
del base_transfer_model_2
del inception_full_fine_tune_model
del base_transfer_model_3
del inception_multiPhase_fine_tune_model
del base_transfer_model_4

In [511]:

gc.collect()
tf.keras.backend.clear_session(
    free_memory=True
)
tf.compat.v1.reset_default_graph()


## Comparative Analysis

[Compare the performance of your model(s) against the baseline model. Discuss any improvements or setbacks and the reasons behind them.]

A table comparing the performances of different models and hyperparameter settings can be found in the github (Model_Performance_overview.xls or Model_Performance_overview.csv).

Some results stand out:

* data augmentation seems to lower model performance across the board even when we see overfitting in training. The likely reason is that the data itself is very regular without a lot of orientation of the features in the images. Therefore, we well adjust data augmentation in future to exlude image flipping etc.
* The transfer learning model performs worse than the 3 relatively simple models. Especially for low image resolutions. The most likely reason is that, as of now we only use feature extraction. For any image size that the model was not originally trained on this will very likely mean a bad performance. For higher resolutions the transfer learning model performs better in comparison
* Higher image resolution does not really improve model performance.

Some things are still missing in the analysis / evaluation and will be added in the near future:

* Transfer learning models with fine tuning
* Different transfer learning base architectures
* When a best model is found we will tackle the task of identifying the drift label class
* More finetuning of hyperparameters for few selected models
* class weighting instead of balanced dataset (balanced dataset is very small)